In [1]:
#The first thing is to import the libraries needed to execute the code successfully
import sqlite3
import pandas as pd
import numpy as np
import re
import datetime
import os
import requests
from bs4 import BeautifulSoup
import matplotlib.pylab as plt
plt.rcdefaults()
%matplotlib inline

In [2]:
#STEP 1: Functions to extract and read tables from the database 
def read_sql_table(path,table):
    df = sqlite3.connect(path)
    df_table = pd.read_sql_query('SELECT * FROM '+table,df)
    return df_table

def read_and_merge(path):
    df_business_info = read_sql_table(path, 'business_info')
    df_personal_info = read_sql_table(path, 'personal_info')
    df_rank_info = read_sql_table(path, 'rank_info')
    df4 = pd.merge(df_business_info, df_personal_info, on=['id', 'id'])
    df_dirt = pd.merge(df4, df_rank_info, on=['id', 'id'])
    df_dirt.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/df_dirt.csv', index=False)
    return df_dirt

In [3]:
#CALLING FUNCTION STEP 1:
read_and_merge('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,4656,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,None,1,bEZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0,52
1,1874,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,None,3,bufFETT,87 years old,54,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
3,2637,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,None,4,arnAULt,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0,55
4,8242,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,None,5,zUCKERBERG,1985,56,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,None,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
2204,3538,2256,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,None,2134,zhOU,55 years old,2256,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0,2256
2205,7797,2257,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,None,1856,ZHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0,2257
2206,3123,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,None,1978,ZHUO,52 years old,2258,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0,2258


In [4]:
#EXTRACTING df_dirt
df_dirt = pd.read_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/df_dirt.csv')
df_dirt

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,4656,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1,bEZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0,52
1,1874,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3,bufFETT,87 years old,54,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
3,2637,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4,arnAULt,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0,55
4,8242,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5,zUCKERBERG,1985,56,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,NaN,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
2204,3538,2256,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,NaN,2134,zhOU,55 years old,2256,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0,2256
2205,7797,2257,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,NaN,1856,ZHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0,2257
2206,3123,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,NaN,1978,ZHUO,52 years old,2258,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0,2258


In [5]:
#STEP 2: Remove columns that do not add value
def drop(df, list_of_columns):
    df = df_dirt.drop(list_of_columns, axis=1)
    return df

In [6]:
#CALLING FUNCTION STEP 2:
columns_to_drop=['id','realTimeWorth','Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','realTimePosition','lastName']
df2 = drop(df_dirt,columns_to_drop)

In [7]:
#STEP 3: WRANGLING: col1-worth, col2-worthchange:
def repla_cols(df, col1, col2):
    df[col1].replace({"BUSD": ' '})
    df[col2].replace('na', '0.0', regex=True, inplace=True)
    df[col1].replace('[BUSD]+$', '', regex=True, inplace=True)
    df[col2].replace('[millions USD]+$', '', regex=True, inplace=True)
    df[col2].astype('float').astype('int')
    df[col2].astype('float').astype('int')
    return df

In [8]:
#CALLING FUNCTION STEP 3:
repla_cols(df2, 'worth', 'worthChange')

,Source,worth,worthChange,age,gender,country,image,name,position
0,Technology ==> Amazon,112.0,0.0,54 years old,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0
1,Technology ==> Microsoft,90.0,-0.001,62 years old,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0
2,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,87 years old,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0
3,Fashion & Retail ==> LVMH,72.0,0.0,69 years old,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0
4,Technology ==> Facebook,71.0,0.0,1985,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0
...,...,...,...,...,...,...,...,...,...
2203,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,51 years old,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0
2204,Manufacturing ==> electrical equipment,1.0,0.0,55 years old,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0
2205,Manufacturing ==> electrical equipment,1.0,0.0,51 years old,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0
2206,Manufacturing ==> printed circuit boards,1.0,0.0,52 years old,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0


In [9]:
#STEP 4: Rename worth and worthChange cols
def rename2(df):
    df2 = df.rename(columns={'worth':'worth BUSD', 'worthChange':' worthChange millions USD'})
    return df2

In [11]:
#CALLING FUNCTION STEP 4:
rename2(df2)

,Source,worth BUSD,worthChange millions USD,age,gender,country,image,name,position
0,Technology ==> Amazon,112.0,0.0,54 years old,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0
1,Technology ==> Microsoft,90.0,-0.001,62 years old,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0
2,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,87 years old,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0
3,Fashion & Retail ==> LVMH,72.0,0.0,69 years old,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0
4,Technology ==> Facebook,71.0,0.0,1985,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0
...,...,...,...,...,...,...,...,...,...
2203,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,51 years old,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0
2204,Manufacturing ==> electrical equipment,1.0,0.0,55 years old,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0
2205,Manufacturing ==> electrical equipment,1.0,0.0,51 years old,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0
2206,Manufacturing ==> printed circuit boards,1.0,0.0,52 years old,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0


In [12]:
#STEP 5: Wrangling gender column 
def wrangling_gender(df, column):
    df[column].replace('[M]+$', 'Male', regex=True, inplace=True)
    df[column].replace('[F]+$', 'Female', regex=True, inplace=True)
    df[column].astype('object').astype('str')
    return df

In [14]:
#CALLING FUNCTION STEP 5:
wrangling_gender(df2, 'gender')

,Source,worth,worthChange,age,gender,country,image,name,position
0,Technology ==> Amazon,112.0,0.0,54 years old,Male,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0
1,Technology ==> Microsoft,90.0,-0.001,62 years old,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0
2,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,87 years old,Male,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0
3,Fashion & Retail ==> LVMH,72.0,0.0,69 years old,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0
4,Technology ==> Facebook,71.0,0.0,1985,Male,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0
...,...,...,...,...,...,...,...,...,...
2203,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,51 years old,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0
2204,Manufacturing ==> electrical equipment,1.0,0.0,55 years old,Male,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0
2205,Manufacturing ==> electrical equipment,1.0,0.0,51 years old,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0
2206,Manufacturing ==> printed circuit boards,1.0,0.0,52 years old,Female,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0


In [18]:
#STEP 6: Wrangling age column
def age_wrangling(df, column):
    df[column].replace('[years old]+$', '', regex=True, inplace=True)      
    df[column].fillna(str(-2019), inplace=True)
    current_year = datetime.datetime.today().year
    age = []
    for i in df[column]:
        if len(str(i)) == 4:
            age.append(str(current_year-int(i)))
        else:
            age.append(i)
                
    df[column] = age
    
    return df

In [19]:
#CALLING FUNCTION STEP 6:
age_wrangling(df2,'age')

,Source,worth,worthChange,age,gender,country,image,name,position
0,Technology ==> Amazon,112.0,0.0,54,Male,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0
1,Technology ==> Microsoft,90.0,-0.001,62,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0
2,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,87,Male,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0
3,Fashion & Retail ==> LVMH,72.0,0.0,69,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0
4,Technology ==> Facebook,71.0,0.0,35,Male,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0
...,...,...,...,...,...,...,...,...,...
2203,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,51,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0
2204,Manufacturing ==> electrical equipment,1.0,0.0,55,Male,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0
2205,Manufacturing ==> electrical equipment,1.0,0.0,51,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0
2206,Manufacturing ==> printed circuit boards,1.0,0.0,52,Female,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0


In [20]:
#STEP 7: Wrangling non values in Country columns
def non_values(df, column):
    df[column].replace('None', np.nan, regex=True, inplace=True)
    df_no_null_values = df[df[column].isnull()] 
    df.drop(df[df[column].isnull()].index, inplace = True)
    return df

In [21]:
#CALLING FUNCTION STEP 7:
non_values(df2, 'country')

,Source,worth,worthChange,age,gender,country,image,name,position
1,Technology ==> Microsoft,90.0,-0.001,62,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0
2,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,87,Male,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0
6,Telecom ==> telecom,67.1,0.001,78,Male,Mexico,https://specials-images.forbesimg.com/imageser...,carlos sLIM HELU,7.0
7,Diversified ==> Koch Industries,60.0,0.0,82,Male,USA,https://specials-images.forbesimg.com/imageser...,chaRLES koch,8.0
18,"Energy ==> petrochemicals, oil & gas",40.1,0.0,61,Male,India,https://specials-images.forbesimg.com/imageser...,mukesH AMBANI,19.0
...,...,...,...,...,...,...,...,...,...
2191,Real Estate ==> real estate,1.0,0.0,73,Male,Australia,https://specials-images.forbesimg.com/imageser...,sam taRASCio,2192.0
2194,Finance and Investments ==> investments,1.0,0.0,63,Male,Norway,https://specials-images.forbesimg.com/imageser...,ALEXANDER VIK,2195.0
2202,Diversified ==> air conditioning,1.0,0.0,53,Male,People's Republic of China,https://specials-images.forbesimg.com/imageser...,yAO XINYi,2203.0
2203,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,51,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0


In [ ]:
#STEP 8: Converting to lowercase:
def lowercase(df):
    df1 = df.apply(lambda x: x.astype(str).str.lower())
    return df1

In [ ]:
#CALLING FUNCTION STEP 8:
lowercase(df2)

In [ ]:
#STEP 9: WEB SCRAPING
url = 'https://en.wikipedia.org/wiki/List_of_countries_and_dependencies_by_population'
def web_scraping(url):
    html = requests.get(url).content
    soup = BeautifulSoup(html, "lxml")
    table = soup.find_all('table',{'class':'wikitable sortable mw-datatable'})[0]
    rows = table.find_all('tr')
    rows_parsed = [row.text for row in rows]
    return rows_parsed

def smart_parser(row_text):
    row_text = row_text.replace('\nPopulation\n\n\n', '\nPopulation\n').strip('\n')
    row_text = row_text.replace('\n\n', '\n').strip('\n')
    row_text = row_text.replace('\nPopulation\n% of World\n', '\nPopulation % of World\n').strip('\n')
    row_text = re.sub('\[\d\]', '', row_text)
    return list(map(lambda x: x.strip(), row_text.split('\n')))

#well_parsed = list(map(lambda x: smart_parser(x), rows_parsed))

#name_columns = well_parsed[0]
#data_scraped = well_parsed[1:]

        #Creating DF:
def scraping_df(data,colnames):
    df_scrapping = pd.DataFrame(data, columns=colnames)
    df_scrapping['Country(or dependent territory)']=df_scrapping['Country(or dependent territory)'].replace(['\[(.*?)\]','\(([^\)]+)\)'], ['',''], regex=True)
    df_scrapping.columns = ['Rank', 'country','Population',' % of World Population','Date','Source']
    return df_scrapping

In [ ]:
#STEP 10: Merging df_scrapping with df1, country column:
def merge_scraping_df(df,df,column):
    df_project2=pd.merge(df1, df, on=[column, column])
    return df

In [ ]:
#STEP11: Graphic1: 'worth BUSD by position - a.k.a. Wealth distribution
def graphic_1(df):
    graphic_1 = df['worth BUSD'].hist(bins=100, figsize=(16,8))
    plt.xlabel('Position')
    plt.ylabel('Wealth')
    plt.title("Wealth distribution.png")
    plt.show()
    plt.close()
    plt.savefig("Wealth distribution.png")

In [1]:
#STEP12: Graphic2: 'Number of billionaires per country'
def graphic_2(df):
    colors = ['grey', 'mediumvioletred', 'gold', 'orangered', 'yellowgreen', 'r', 'skyblue', 'navy', 'grey', 'black']
    df['country'].value_counts()[:15].plot(kind='bar', width=0.9, figsize=(20, 10), color=colors)
    plt.xlabel('Country (Top 30)')
    plt.ylabel('Number of billionaires')
    plt.title('Number of billionaires per country')
    plt.savefig("Billionaires - Countries")
    plt.savefig("Number of billionaires per country")
    plt.show()
    plt.close()
    plt.savefig("Number of billionaires per country.png")